In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [AutoML] Damaged car 모델 prediction Sample 코드

AutoML을 통해 생성된 모델을 활용하여 Prediction을 위한 샘플 코드입니다.
Deploy 된 모델의 endpoint를 활용

In [ ]:
import base64
import os, sys

from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict

In [ ]:
PROJECT = "" ## 생성된 모델이 포함되어 있는 Google Cloud Project ID
ENDPOINT_ID = "" ## Deploy된 모델의 Endpoint
FILENAME = "" ## Prediction 하기 위한 파일명
LOCATION = "" ## 모델이 포함된 Google Cloud의 리전 
API_ENDPOINT = "" ## AI Platform 호출을 위한 API endpoint "<region>-aiplatform.googleapis.com"

# The AI Platform services require regional API endpoints.
client_options = {"api_endpoint": API_ENDPOINT}
# Initialize client that will be used to create and send requests.
# This client only needs to be created once, and can be reused for multiple requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
with open(FILENAME, "rb") as f:
    file_content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
encoded_content = base64.b64encode(file_content).decode("utf-8")

instance = predict.instance.ImageClassificationPredictionInstance(
    content=encoded_content,
).to_value()
instances = [instance]
# See gs://google-cloud-aiplatform/schema/predict/params/image_classification_1.0.0.yaml for the format of the parameters.
parameters = predict.params.ImageClassificationPredictionParams(
    confidence_threshold=0.5, max_predictions=5,
).to_value()
endpoint = client.endpoint_path(
    project=PROJECT, location=LOCATION, endpoint=ENDPOINT_ID
)
response = client.predict(
    endpoint=endpoint, instances=instances, parameters=parameters
)
print ("response")
print(" deployed_model_id:", response.deployed_model_id)
# See gs://google-cloud-aiplatform/schema/predict/prediction/image_classification_1.0.0.yaml for the format of the predictions.
predictions = response.predictions
for prediction in predictions:
    print(" prediction:", dict(prediction))